# Obestiy Estimation - Feature Enginerring and Machine Learning Modelling

### In this notebook we will implement various Machine Learning Classification alogrithms to select the best performing model
1) Decision Tree </br>
2) Random Forest</br>
3) KNN </br>
4) XGBClassifier</br>


### Following steps will be taken to do that.
##### Split the Dataset First:
Split the dataset into training and test sets before applying transformations to avoid data leakage.
##### One-Hot Encoding:
Apply one-hot encoding only to the training set using fit_transform.</br>
Use the same encoder to transform the test set with transform, ensuring consistency.</br>
Handle unknown categories with handle_unknown='ignore'.
##### Label Encoding for Target Variable:
Apply label encoding to the entire target column (train + test) for consistent label mappings across splits.
Safe because it doesn’t leak information from features to the model.
##### Feature Scaling:
Apply scaling (e.g., StandardScaler or MinMaxScaler) only to the training set using fit_transform.
Use the same scaler to transform the test set with transform.
##### Save Encoders/Scalers:
Save the fitted encoders and scalers to ensure consistent transformations for future data.

#####  GridsearchCV and 5-fold cross-validation to compare the accuracies.

In [87]:
import pandas as pd

In [88]:
# Read data and convert to a dataframe
clean_data_df = pd.read_csv(r'../data/clean_data.csv')

clean_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2087 entries, 0 to 2086
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Gender                     2087 non-null   object 
 1   Age                        2087 non-null   float64
 2   Height                     2087 non-null   float64
 3   Weight                     2087 non-null   float64
 4   Family_History             2087 non-null   object 
 5   High_Cal_Foods_Frequently  2087 non-null   object 
 6   Freq_Veg                   2087 non-null   float64
 7   Num_Meals                  2087 non-null   float64
 8   Snacking                   2087 non-null   object 
 9   Smoke                      2087 non-null   object 
 10  Water_Intake               2087 non-null   float64
 11  Calorie_Monitoring         2087 non-null   object 
 12  Phys_Activity              2087 non-null   float64
 13  Tech_Use                   2087 non-null   float

In [89]:
# Feature columns by preprocessing type: categorical, or continous

cat_cols = ['Gender', 'Family_History', 'High_Cal_Foods_Frequently', 'Snacking','Smoke', 'Calorie_Monitoring', 'Freq_Alcohol', 'Transportation']

num_cols = ['Age', 'Height', 'Weight', 'Freq_Veg', 'Num_Meals','Water_Intake', 'Phys_Activity', 'Tech_Use']

### Get X and y

In [90]:
X = clean_data_df.drop('Obesity_Level',axis=1)  
y = clean_data_df['Obesity_Level'] 

X.shape, y.shape

((2087, 16), (2087,))

### Train test split - stratified splitting
Stratified splitting means that when you generate a training / validation dataset split, it will attempt to keep the same percentages of classes in each split.

These dataset divisions are usually generated randomly according to a target variable. However, when doing so, the proportions of the target variable among the different splits can differ, especially in the case of small datasets.

In [91]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,stratify=y,test_size=0.2,random_state=42)

In [92]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1669, 16), (418, 16), (1669,), (418,))

### One hot Encoding

In [93]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
# Encoding multiple columns. 
transformer = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore'), 
     cat_cols))

transformer

ColumnTransformer(transformers=[('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['Gender', 'Family_History',
                                  'High_Cal_Foods_Frequently', 'Snacking',
                                  'Smoke', 'Calorie_Monitoring', 'Freq_Alcohol',
                                  'Transportation'])])

### X_train encoding

In [94]:
# Transforming
transformed = transformer.fit_transform(X_train)
# Transformating back
transformed_df = pd.DataFrame(transformed, columns=transformer.get_feature_names_out())
# One-hot encoding removed an index. Let's put it back:
transformed_df.index = X_train.index
# Joining tables
X_train = pd.concat([X_train, transformed_df], axis=1)
# Dropping old categorical columns
X_train.drop(cat_cols, axis=1, inplace=True)
# CHecking result
X_train.head()

,Age,Height,Weight,Freq_Veg,Num_Meals,Water_Intake,Phys_Activity,Tech_Use,onehotencoder__Gender_Female,onehotencoder__Gender_Male,...,onehotencoder__Calorie_Monitoring_yes,onehotencoder__Freq_Alcohol_Always,onehotencoder__Freq_Alcohol_Frequently,onehotencoder__Freq_Alcohol_Sometimes,onehotencoder__Freq_Alcohol_no,onehotencoder__Transportation_Automobile,onehotencoder__Transportation_Bike,onehotencoder__Transportation_Motorbike,onehotencoder__Transportation_Public_Transportation,onehotencoder__Transportation_Walking
1549,25.883749,1.767077,114.133149,2.225731,3.000000,2.154326,1.266866,0.390178,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1574,30.686701,1.644517,100.004418,2.964050,2.123138,1.000000,1.699181,0.620465,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1155,47.706100,1.743935,84.729197,2.535315,3.000000,1.146595,0.313810,0.000000,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
610,22.998709,1.740108,53.657270,2.241606,3.000000,1.846626,2.460238,0.814518,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
906,21.009437,1.606810,67.773914,2.000000,3.156309,3.000000,1.179592,0.086868,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


### X_test encoding

In [95]:
# Transforming
transformed = transformer.transform(X_test)
# Transformating back
transformed_df = pd.DataFrame(transformed, columns=transformer.get_feature_names_out())
# One-hot encoding removed an index. Let's put it back:
transformed_df.index = X_test.index
# Joining tables
X_test = pd.concat([X_test, transformed_df], axis=1)
# Dropping old categorical columns
X_test.drop(cat_cols, axis=1, inplace=True)
# CHecking result
X_test.head()

,Age,Height,Weight,Freq_Veg,Num_Meals,Water_Intake,Phys_Activity,Tech_Use,onehotencoder__Gender_Female,onehotencoder__Gender_Male,...,onehotencoder__Calorie_Monitoring_yes,onehotencoder__Freq_Alcohol_Always,onehotencoder__Freq_Alcohol_Frequently,onehotencoder__Freq_Alcohol_Sometimes,onehotencoder__Freq_Alcohol_no,onehotencoder__Transportation_Automobile,onehotencoder__Transportation_Bike,onehotencoder__Transportation_Motorbike,onehotencoder__Transportation_Public_Transportation,onehotencoder__Transportation_Walking
1153,19.955257,1.589100,72.713611,3.00000,3.856434,2.000000,1.324170,1.000000,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
132,30.000000,1.770000,109.000000,3.00000,3.000000,1.000000,2.000000,0.000000,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1923,20.601222,1.738717,128.114161,3.00000,3.000000,1.797041,1.427413,0.966181,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
846,16.950499,1.603501,65.000000,2.96008,1.000000,2.000000,0.736032,1.344072,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1246,29.506287,1.826970,108.751502,2.00000,2.877583,2.358038,0.877295,1.817146,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [96]:
print(X_train.columns)

Index(['Age', 'Height', 'Weight', 'Freq_Veg', 'Num_Meals', 'Water_Intake',
       'Phys_Activity', 'Tech_Use', 'onehotencoder__Gender_Female',
       'onehotencoder__Gender_Male', 'onehotencoder__Family_History_no',
       'onehotencoder__Family_History_yes',
       'onehotencoder__High_Cal_Foods_Frequently_no',
       'onehotencoder__High_Cal_Foods_Frequently_yes',
       'onehotencoder__Snacking_Always', 'onehotencoder__Snacking_Frequently',
       'onehotencoder__Snacking_Sometimes', 'onehotencoder__Snacking_no',
       'onehotencoder__Smoke_no', 'onehotencoder__Smoke_yes',
       'onehotencoder__Calorie_Monitoring_no',
       'onehotencoder__Calorie_Monitoring_yes',
       'onehotencoder__Freq_Alcohol_Always',
       'onehotencoder__Freq_Alcohol_Frequently',
       'onehotencoder__Freq_Alcohol_Sometimes',
       'onehotencoder__Freq_Alcohol_no',
       'onehotencoder__Transportation_Automobile',
       'onehotencoder__Transportation_Bike',
       'onehotencoder__Transportation_Moto

In [97]:
# Setting new feature names

X_train.columns = ['Age', 'Height', 'Weight', 'Freq_Veg', 'Num_Meals', 'Water_Intake',
       'Phys_Activity', 'Tech_Use', 'Gender_Female',
       'Gender_Male', 'Family_History_no',
       'Family_History_yes',
       'High_Cal_Foods_Frequently_no',
       'High_Cal_Foods_Frequently_yes',
       'Snacking_Always', 'Snacking_Frequently',
       'Snacking_Sometimes', 'Snacking_no',
       'Smoke_no', 'Smoke_yes',
       'Calorie_Monitoring_no',
       'Calorie_Monitoring_yes',
       'Freq_Alcohol_Always',
       'Freq_Alcohol_Frequently',
       'Freq_Alcohol_Sometimes',
       'Freq_Alcohol_no',
       'Transportation_Automobile',
       'Transportation_Bike',
       'Transportation_Motorbike',
       'Transportation_Public_Transportation',
       'Transportation_Walking']

X_test.columns = ['Age', 'Height', 'Weight', 'Freq_Veg', 'Num_Meals', 'Water_Intake',
       'Phys_Activity', 'Tech_Use', 'Gender_Female',
       'Gender_Male', 'Family_History_no',
       'Family_History_yes',
       'High_Cal_Foods_Frequently_no',
       'High_Cal_Foods_Frequently_yes',
       'Snacking_Always', 'Snacking_Frequently',
       'Snacking_Sometimes', 'Snacking_no',
       'Smoke_no', 'Smoke_yes',
       'Calorie_Monitoring_no',
       'Calorie_Monitoring_yes',
       'Freq_Alcohol_Always',
       'Freq_Alcohol_Frequently',
       'Freq_Alcohol_Sometimes',
       'Freq_Alcohol_no',
       'Transportation_Automobile',
       'Transportation_Bike',
       'Transportation_Motorbike',
       'Transportation_Public_Transportation',
       'Transportation_Walking']
# After renaming the columns

X_train.head()


,Age,Height,Weight,Freq_Veg,Num_Meals,Water_Intake,Phys_Activity,Tech_Use,Gender_Female,Gender_Male,...,Calorie_Monitoring_yes,Freq_Alcohol_Always,Freq_Alcohol_Frequently,Freq_Alcohol_Sometimes,Freq_Alcohol_no,Transportation_Automobile,Transportation_Bike,Transportation_Motorbike,Transportation_Public_Transportation,Transportation_Walking
1549,25.883749,1.767077,114.133149,2.225731,3.000000,2.154326,1.266866,0.390178,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1574,30.686701,1.644517,100.004418,2.964050,2.123138,1.000000,1.699181,0.620465,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1155,47.706100,1.743935,84.729197,2.535315,3.000000,1.146595,0.313810,0.000000,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
610,22.998709,1.740108,53.657270,2.241606,3.000000,1.846626,2.460238,0.814518,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
906,21.009437,1.606810,67.773914,2.000000,3.156309,3.000000,1.179592,0.086868,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


### Feature scaling

In [98]:
from sklearn.preprocessing import StandardScaler
Standard_Scaler = StandardScaler()
x_scaled = Standard_Scaler.fit_transform(X_train[num_cols])
x_test_scaled = Standard_Scaler.transform(X_test[num_cols])

### Apply Label encoder

In [99]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

y_train

1549        Obesity_Type_II
1574        Obesity_Type_II
1155    Overweight_Level_II
610     Insufficient_Weight
906      Overweight_Level_I
               ...         
176           Normal_Weight
985     Overweight_Level_II
1564        Obesity_Type_II
128           Normal_Weight
1141    Overweight_Level_II
Name: Obesity_Level, Length: 1669, dtype: object

### Classifier Models 

In [100]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

In [101]:
models={'RandomForest':RandomForestClassifier(),
        'DecisionTree':DecisionTreeClassifier(),
        'KNeighbors':KNeighborsClassifier(),
        'xgbc': XGBClassifier()}

In [106]:
param_grids={'RandomForest': {
                'n_estimators': [100, 200],
                'max_depth': [None, 10, 20]},
            'DecisionTree': {
                'max_depth': [None, 10, 20],
                'min_samples_split': [2, 5, 10]
            },'KNeighbors':{
            'n_neighbors': [3, 5, 7]
        }, 'xgbc':{
            'n_estimators':[600],
            'learning_rate':[0.03],
            'objective':['multi:softmax'], 
            'verbosity':[0], 
            'nthread':[-1], 
            'random_state':[42]
        }
    }

In [107]:
from sklearn.model_selection import GridSearchCV

best_models={}

for model in models:
    gsv=GridSearchCV(estimator=models[model],param_grid=param_grids[model],cv=10,scoring='accuracy')
    gsv.fit(x_scaled,y_train_encoded)
    best_models[model]=gsv.best_estimator_
   
    print(f'Best parameters for {model} is {gsv.best_params_}.')
    print(f'Best score for {model} is {gsv.best_score_}.')

Best parameters for RandomForest is {'max_depth': 20, 'n_estimators': 200}.
Best score for RandomForest is 0.9502705432508478.
Best parameters for DecisionTree is {'max_depth': None, 'min_samples_split': 2}.
Best score for DecisionTree is 0.9322956496645265.
Best parameters for KNeighbors is {'n_neighbors': 3}.
Best score for KNeighbors is 0.8178738907726716.
Best parameters for xgbc is {'learning_rate': 0.03, 'n_estimators': 600, 'nthread': -1, 'objective': 'multi:softmax', 'random_state': 42, 'verbosity': 0}.
Best score for xgbc is 0.9634441959454584.
